In [1]:
from __future__ import print_function
from __future__ import division

from sklearn.datasets import make_classification
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Load data set and target values
data, target = make_classification(
    n_samples=1000,
    n_features=45,
    n_informative=12,
    n_redundant=7
)

In [3]:
def svccv(C, gamma):
    val = cross_val_score(
        SVC(C=C, gamma=gamma, random_state=2),
        data, target, 'f1', cv=2
    ).mean()

    return val

def rfccv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RFC(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            random_state=2
        ),
        data, target, 'f1', cv=2
    ).mean()
    return val

In [4]:
gp_params = {"alpha": 1e-5}

svcBO = BayesianOptimization(svccv,
        {'C': (0.001, 100), 'gamma': (0.0001, 0.1)})
svcBO.explore({'C': [0.001, 0.01, 0.1], 'gamma': [0.001, 0.01, 0.1]})

rfcBO = BayesianOptimization(
        rfccv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999)}
    )




In [5]:
svcBO.maximize(n_iter=10, **gp_params)
print('-' * 53)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    1 | 00m00s |    0.66755 |    0.0010 |    0.0010 | 
    2 | 00m00s |    0.66755 |    0.0100 |    0.0100 | 
    3 | 00m00s |    0.66755 |    0.1000 |    0.1000 | 
    4 | 00m00s |    0.88966 |   38.6644 |    0.0516 | 
    5 | 00m00s |    0.71895 |   33.6271 |    0.0917 | 
    6 | 00m00s |    0.70555 |   63.2100 |    0.0955 | 
    7 | 00m00s |    0.83455 |   18.6330 |    0.0700 | 
    8 | 00m00s |    0.88678 |   96.6106 |    0.0529 | 
Bayesian Optimization
-----------------------------------------------------
 Step |   Time |      Value |         C |     gamma | 
    9 | 00m08s |    0.86706 |   82.6372 |    0.0001 | 
   10 | 00m05s |    0.85752 |   49.8801 |    0.0001 | 
   11 | 00m04s |    0.84436 |   10.1086 |    0.0001 | 
   12 | 00m04s |    0.68937 |   90.1402 |    0.1000 | 
   13 | 00m05s |    0.86147 |   74.0137 |    0.0001 | 
   14 | 00m04s |    0.68937 | 

In [6]:
rfcBO.maximize(n_iter=10, **gp_params)
print('-' * 53)

Initialization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    1 | 00m00s |    0.86383 |         0.4437 |              3.0557 |        64.1150 | 
    2 | 00m02s |    0.86096 |         0.7809 |             20.4751 |       112.6625 | 
    3 | 00m01s |    0.87683 |         0.3851 |              6.7568 |       124.3749 | 
    4 | 00m03s |    0.86259 |         0.7705 |             17.6409 |       176.3991 | 
    5 | 00m00s |    0.85980 |         0.5737 |             16.2066 |        49.6330 | 
Bayesian Optimization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    6 | 00m25s |    0.85448 |         0.9990 |              2.0000 |       250.0000 | 
    7 | 00m16s |    0.86803 |         0.7655 |              2.0008 |       137.1176 | 
    8 | 

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.71351263e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


   12 | 00m14s |    0.87510 |         0.1637 |              2.0758 |       154.8331 | 
   13 | 00m13s |    0.89000 |         0.1211 |              2.0348 |       148.2286 | 
   14 | 00m06s |    0.75462 |         0.1039 |              2.5232 |        10.4599 | 
   15 | 00m09s |    0.87487 |         0.2349 |             24.8816 |       249.7517 | 
-----------------------------------------------------


In [7]:
print('Final Results')
print('SVC: %f' % svcBO.res['max']['max_val'])
print('RFC: %f' % rfcBO.res['max']['max_val'])

Final Results
SVC: 0.889663
RFC: 0.889995
